In [98]:
with open ('gita.txt') as f:
   content=f.read()

In [99]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle

In [100]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 500,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)

In [101]:
content

' I\nDhritirashtra:\nRanged thus for battle on the sacred plain--\nOn Kurukshetra--say, Sanjaya! say\nWhat wrought my people, and the Pandavas?\n\nSanjaya:\nWhen he beheld the host of Pandavas,\nRaja Duryodhana to Drona drew,\nAnd spake these words: "Ah, Guru! see this line,\nHow vast it is of Pandu fighting-men,\nEmbattled by the son of Drupada,\nThy scholar in the war! Therein stand ranked\nChiefs like Arjuna, like to Bhima chiefs,\nBenders of bows; Virata, Yuyudhan,\nDrupada, eminent upon his car,\nDhrishtaket, Chekitan, Kasi\'s stout lord,\nPurujit, Kuntibhoj, and Saivya,\nWith Yudhamanyu, and Uttamauj\nSubhadra\'s child; and Drupadi\'s;-all famed!\nAll mounted on their shining chariots!\nOn our side, too,--thou best of Brahmans! see\nExcellent chiefs, commanders of my line,\nWhose names I joy to count: thyself the first,\nThen Bhishma, Karna, Kripa fierce in fight,\nVikarna, Aswatthaman; next to these\nStrong Saumadatti, with full many more\nValiant and tried, ready this day to di

In [102]:
texts=text_splitter.split_text(content)

Created a chunk of size 851, which is longer than the specified 500
Created a chunk of size 619, which is longer than the specified 500
Created a chunk of size 515, which is longer than the specified 500


In [103]:
len(texts)

299

In [61]:
file_path="/Users/aditya/Documents/splitted_text.txt"
with open(file_path, "w") as f:
    f.write(str(texts))

In [104]:
model =SentenceTransformer("all-mpnet-base-v2")
vectors=model.encode(texts)

In [105]:
vectors

array([[ 0.03738715, -0.05641272, -0.03524848, ...,  0.06963781,
        -0.03225681, -0.0466377 ],
       [-0.01605082, -0.04805276, -0.00923893, ...,  0.05581874,
        -0.01455058, -0.05702751],
       [-0.04353648, -0.09105222, -0.01862514, ..., -0.00175901,
         0.00063855, -0.01565077],
       ...,
       [ 0.06616037,  0.05158279, -0.02120283, ...,  0.03294563,
        -0.01513285, -0.03993101],
       [ 0.08957193,  0.01219572, -0.0088336 , ..., -0.03399097,
         0.0647816 , -0.0187907 ],
       [ 0.05677417,  0.07327044,  0.00011075, ...,  0.01346605,
         0.02066537, -0.0170597 ]], dtype=float32)

In [106]:
vectors.shape[1]

768

In [107]:
index=faiss.IndexFlatL2(vectors.shape[1])

In [108]:
index.add(vectors)

In [109]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x29c80cbd0> >

In [110]:
search="elon"
vec=model.encode(search)

In [111]:
svec=np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [112]:
index.search(svec,k=2)

(array([[1.4891968, 1.5478956]], dtype=float32), array([[253, 254]]))

In [91]:
from langchain.chains import RetrievalQAWithSourcesChain 

In [31]:
from apikey import api
import os 
from langchain.llms import OpenAI
os.environ['OPENAI_API_KEY'] = api
llm = OpenAI(temperature=0.1)
ans=llm(texts[210]+texts[224]+'using this texts tell me whats the use of Truth? Think you are Lord Krishna answer like him')

In [32]:
ans

'\n\nThe use of Truth is to bring us closer to the Divine. By understanding and embracing the Truth, we can come to know the One Inseparable, the changeless Life that is present in all things. We can come to understand the mystery of the universe and our place in it. By clinging to Truth, we can come to Me and find solace and comfort in My love.'

In [113]:
file_path="/Users/aditya/Documents/vector.pkl"
with open(file_path, "wb") as f:
    pickle.dump (index, f)

In [14]:
import os
import pickle

In [15]:
file_path="/Users/aditya/Documents/vector.pkl"
if os. path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex= pickle.load(f)

In [27]:
model =SentenceTransformer("all-mpnet-base-v2")
search="krishna whats meditationh"
vec=model.encode(search)
svec=np.array(vec).reshape(1,-1)
x=vectorIndex.search(svec,k=2)